## Packages needed

In [1]:
import geopandas as gpd
import os
from rasterstats import zonal_stats
import numpy as np
import tkinter as tk
from tkinter import filedialog, messagebox
import gdal
import rasterio
import ogr
import warnings
import json
import pandas as pd
from earthpy import clip
from shapely.geometry import JOIN_STYLE

import scipy.spatial
from pathlib import Path
warnings.filterwarnings('ignore')

root = tk.Tk()
root.withdraw()
root.attributes("-topmost", True)

''

## Clip raster

In [ ]:
def clipRaster(raster, polygon, xRes, yRes, output):
    kwargs = {'xRes': xRes, 'yRes': yRes, 'dstNodata':0} 
    warped = gdal.Warp(output, raster, cutlineDSName=polygon, **kwargs)

## Reclassify raster

In [1]:
def reclassifyRasters(file, opt, workspace):
    driver = gdal.GetDriverByName("GTiff")
    band = file.GetRasterBand(1)
    lista = band.ReadAsArray()

    for j in range(file.RasterXSize):
        for i in range(file.RasterYSize):
            if lista[i,j] < 999999999 and lista[i,j] > 0:
                lista[i,j] = 1


    if opt == 1:
        file2 = driver.Create(workspace + r"/reclassifiedPop.tif", file.RasterXSize , file.RasterYSize , 1)
    else:
        file2 = driver.Create(workspace + r"/reclassifiedNTL.tif", file.RasterXSize , file.RasterYSize , 1)
        
    file2.GetRasterBand(1).WriteArray(lista)

    proj = file.GetProjection()
    georef = file.GetGeoTransform()
    file2.SetProjection(proj)
    file2.SetGeoTransform(georef)
    file2.FlushCache()

## Convert raster to polygon

In [5]:
def toPolygon(source, opt, workspace):
    sourceRaster = gdal.Open(source)
    band = sourceRaster.GetRasterBand(1)
    bandArray = band.ReadAsArray()
    
    if opt == 1:
        outShapefile = workspace + r"\clusters"
    else:
        outShapefile = workspace + r"\NTLArea"
        
    driver = ogr.GetDriverByName("ESRI Shapefile")


    if os.path.exists(outShapefile+".shp"):
        driver.DeleteDataSource(outShapefile+".shp")

    outDatasource = driver.CreateDataSource(outShapefile+ ".shp")

    outLayer = outDatasource.CreateLayer(outShapefile+ ".shp", srs=None)
    newField = ogr.FieldDefn('PLACEHOLDER', ogr.OFTInteger)
    outLayer.CreateField(newField)
    gdal.Polygonize( band, None, outLayer, 0, [], callback=None )
    outDatasource.Destroy()
    sourceRaster = None
    
    if opt == 2:
        NTLArea=gpd.read_file(workspace + r"/NTLArea.shp")
        clean = NTLArea[NTLArea.PLACEHOLDE != 0]
        clean_b = clean.buffer(0)
        clean_b.to_file(workspace + r"/NTLArea.shp")

## Buffering

In [6]:
def buffer(polygons, workspace):
    clusters=gpd.read_file(polygons)
    clean = clusters[clusters.PLACEHOLDE != 0]
    buffered = clean.buffer(0.001)
    convex_hull = buffered.convex_hull
    buffered.to_file(workspace + "/clusters.shp")
    convex_hull = gpd.read_file(workspace + r"/clusters.shp")
    convex_hull["fid"] = 1
    dissolved=convex_hull.dissolve(by="fid")
    dissolved.to_file(workspace + r"/clusters.shp")
    return dissolved

## Convert multipart ot singlepart

In [7]:
def multi2single(gpdf):
    gpdf_singlepoly = gpdf[gpdf.geometry.type == 'Polygon']
    gpdf_multipoly = gpdf[gpdf.geometry.type == 'MultiPolygon']

    for i, row in gpdf_multipoly.iterrows():
        Series_geometries = pd.Series(row.geometry)
        df = pd.concat([gpd.GeoDataFrame(row, crs=gpdf_multipoly.crs).T]*len(Series_geometries), ignore_index=True)
        df['geometry']  = Series_geometries
        gpdf_singlepoly = pd.concat([gpdf_singlepoly, df])

    gpdf_singlepoly.reset_index(inplace=True, drop=True)
    return gpdf_singlepoly

## Dissolve and split

In [8]:
def dissolveandsplit(inputfile, crs, filename_admin, country_name, workspace): 
    inputs = gpd.read_file(inputfile)
    eps = 0.001
    dissolved = inputs.buffer(eps, 1, join_style=JOIN_STYLE.mitre).buffer(-eps, 1, join_style=JOIN_STYLE.mitre)
    dissolved.to_file(workspace + r"\clusters.shp")
    dissolved = gpd.read_file(workspace + r"\clusters.shp")

    admin = gpd.read_file(filename_admin)
    lines = admin.boundary
    dissolved_clip = clip.clip_shp(dissolved,admin)
    buffered_lines = lines.buffer(0.000000001)
    buffered_lines.to_file(workspace + r"\bufferedlines.shp")
    buffered_lines = gpd.read_file(workspace + r"\bufferedlines.shp")

    clusters = gpd.overlay(dissolved_clip, buffered_lines, how='difference')
    clusters = clusters["geometry"] 
    clusters.to_file(workspace + r"\clusters.shp")
    clusters = gpd.read_file(workspace + r"\clusters.shp")
    clusters['id'] = np.arange(len(clusters))

    clusters.crs = {'init' :'epsg:4326'}
    clusters_proj = clusters.to_crs({ 'init': crs})
    clusters_proj["GridCellAr"] = clusters_proj.area/1000000
    clusters_proj["Country"] = country_name
    clusters = clusters_proj.to_crs({ 'init': 'epsg:4326'})
    return clusters

## Populating clusters

In [9]:
def populatingClusters(clusters,raster,column,method):
    clusters = zonal_stats(
    clusters,
    raster.name,
    stats=[method],
    prefix=column, geojson_out=True, all_touched=True)
    
    return clusters

## Finalizing clusters

In [10]:
def finalizing_clusters(clusters, workspace):
    output = workspace + r'\placeholder.geojson'
    with open(output, "w") as dst:
        collection = {
            "type": "FeatureCollection",
            "features": list(clusters)}
        dst.write(json.dumps(collection))
  
    clusters = gpd.read_file(output)
    clusters.fillna(0, inplace=True)
    os.remove(output)
    clusters.to_file(workspace + r"\clusters.shp")

    dir_name = workspace
    test = os.listdir(dir_name)

    for item in test:
        if item.endswith(".tif") or item.startswith("NTLArea") or item.startswith("bufferedlines"):
            os.remove(os.path.join(dir_name, item))